* count file statistics for Pcap、honepot

In [2]:
import pickle
import numpy as np
import pandas as pd
import os,sys
import random
from tqdm import tqdm

In [8]:
root_dir = "../data_hdd4t/"
hp_date_first = True #先分日期才分四種hp?
pcap_date_dir = False #pcap資料夾有多一層日期資料夾?

找當前root_dir中的所有電信商最大範圍日期做為基準
* list

In [38]:
isps = next(os.walk(root_dir))[1]
if 'csv' in isps:
    isps.remove('csv')
if 'meta_data' in isps:
    isps.remove('meta_data')
# pcap date
all_dates = []
for isp in isps:
#     ori_dirs = next(os.walk(root_dir + isp))[1] #honeypot/pcap
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    if pcap_date_dir == False:
        pcap_files = next(os.walk(pcap_dir))[2]
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        all_dates.extend(dates)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1] #csv/meta_data need to be excluded
        all_dates.extend(pcap_dates)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        all_dates.extend(dates)
    else:
        hp_types_dir = next(os.walk(hp_dir))[1]
        for hp_type in hp_types_dir:
            hp_dates = next(os.walk(hp_dir + hp_type + '/'))[1]
            all_dates.extend(hp_dates)
        
#     for o_d in ori_dirs:
all_dates = sorted(list(set(all_dates)))
all_dates

['2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11']

In [36]:
df = pd.DataFrame(columns=['ISP', 'pcap_loss_date',
                           'amun_loss_date','cowrie_loss_date','dionaea_loss_date','glastopf_loss_date'])
for isp in isps:
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    pcap_loss = []
    amun_loss = []
    cowire_loss = []
    dionaea_loss = []
    glastopf_loss = []
    if pcap_date_dir == False:
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        for a_date in all_dates:
            if a_date not in dates:
                pcap_loss.append(a_date)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1]
        for a_date in all_dates:
            if a_date not in pcap_dates:
                pcap_loss.append(a_date)
        for date in pcap_dates:
            if len(os.listdir(pcap_dir+date) ) == 0:
                pcap_loss.append(date)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        for a_date in all_dates:
            if a_date not in dates:
#                 print(isp)
                amun_loss.append(a_date)
                cowire_loss.append(a_date)                        
                dionaea_loss.append(a_date)                        
                glastopf_loss.append(a_date)
        for date in hp_dates:
            try:
                if len(os.listdir(hp_dir+date+'/amun') ) == 0:
                    amun_loss.append(date)
            except FileNotFoundError:
                amun_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/cowrie') ) == 0:
                    cowire_loss.append(date)
            except FileNotFoundError:
                cowire_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/dionaea') ) == 0:
                    dionaea_loss.append(date)
            except FileNotFoundError:
                dionaea_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/glastopf') ) == 0:
                    glastopf_loss.append(date)
            except FileNotFoundError:
                glastopf_loss.append(date)
    else:
        for a_date in all_dates:
            try:
                if len(os.listdir(hp_dir+'amun/'+a_date) ) == 0:
                    amun_loss.append(a_date)
            except FileNotFoundError:
                amun_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'cowrie/'+a_date) ) == 0:
                    cowire_loss.append(a_date)
            except FileNotFoundError:
                cowire_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'dionaea/'+a_date) ) == 0:
                    dionaea_loss.append(a_date)
            except FileNotFoundError:
                dionaea_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'glastopf/'+a_date) ) == 0:
                    glastopf_loss.append(a_date)
            except FileNotFoundError:
                glastopf_loss.append(a_date)
    temp = [isp, "、".join(pcap_loss), 
            "、".join(amun_loss),"、".join(cowire_loss),"、".join(dionaea_loss),"、".join(glastopf_loss)]
    temp = pd.Series(temp,index=df.columns)
    df = df.append(temp, ignore_index=True)
                
df


亞太
亞太
亞太
亞太
中嘉寬頻
中嘉寬頻
中嘉寬頻
中嘉寬頻
中嘉寬頻
中嘉寬頻
中嘉寬頻
凱擘
凱擘
凱擘
凱擘
台灣之星
台灣之星
台灣之星
台灣之星
台灣之星
遠傳
遠傳
遠傳
遠傳
台哥大
台哥大
台哥大
台哥大
台基開發
台基開發
台基開發
台基開發
中華電信
中華電信
中華電信
中華電信
中華電信
中華電信
台灣固網
台灣固網
台灣固網
台灣固網
台固媒體
台固媒體
台固媒體
台固媒體


,ISP,pcap_loss_date,amun_loss_date,cowrie_loss_date,dionaea_loss_date,glastopf_loss_date
0,亞太,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
1,台灣碩網,2020_01_01、2020_01_02、2020_01_03、2020_01_04,,,,2020-01-02
2,中嘉寬頻,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
3,凱擘,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
4,台灣之星,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
5,遠傳,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
6,台哥大,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
7,台基開發,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
8,中華電信,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...
9,台灣固網,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04


In [37]:
df.iloc[5]

ISP                                                                  遠傳
pcap_loss_date              2020_01_01、2020_01_02、2020_01_03、2020_01_04
amun_loss_date        2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
cowrie_loss_date            2020_01_01、2020_01_02、2020_01_03、2020_01_04
dionaea_loss_date     2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
glastopf_loss_date    2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
Name: 5, dtype: object

In [24]:
'1' in '1'

True